# Arrhythmia Analysis

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In a medical experiment, False Negative is not a desirable outcome. False-negative: A result that appears negative when it should not. An example of a false negative would be if a particular test designed to detect arrhythmia returns a negative result but the person actually does have an arrhythmia. 
This kind of situation can be fatal for the patients, as a false negative diagnosis stop them to refer for further examination. Therefore a correct classification is very much necessary. 

Since this is a supervised classification problem there are a few machine learning algorithm to adapt from such as Decision Tree, Naive Bayes, K Nearest Neighbour and Random Forest etc.

The Naive Bayes approach is based on the idea that the features are conditionally independent and also the features are normally distributed, a quick test shows most of the features are not normally distributed. Keep these in mind we will exclude Naive Bayes as our machine learning algorithm.

K nearest neighbour or KNN another classifier algorithm. The method requires us to develop a similarity or distance on the feature space usually on the basis of a set of predictors or features. Once the distance space is defined, we determine the class by finding the nearest neighbour in the feature space and using that label. Because this model is quite susceptible to noisy data, it is a better idea to find a set of k for k neighbours and then have them vote on the outcome. The value of K has a big impact on the model, if k is small then the classification will be highly impacted by local neighbourhood in contrast, if K is too big then it will respond to most like the response of the population. As a result, normalization is a better choice for KNN as it creates for each variable.

As we know many are better than one and that is the concept of behind the Random Forest algorithm. This is an ensemble learning model that creates multiple decision trees then it aggregates the votes from different decision trees to decide the final class of the test object. The advantages of the Random Forest algorithm are many folds;

 - it is one of the most accurate machines learning algorithm
 - it runs quite efficiently even for very large dataset
 - it can handle quite a complex dataset
 
Although on the downside,

 - Random Forest has the tendency of over-fitting of the model
 - Random Forest is biased in the case of an imbalanced data set.

Because of all these reasons, I will choose the Random Forest as our machine learning algorithm. And We will use Python Scikit-Learn library for that purpose. 

Here, we want to do some preliminary data exploration.

In [ ]:
# Creating a list for the column name
col = [line.strip() for line in open("arrhythmia_column.names", 'r')]

# Creating a dataframe for the data file and the column name from the list.
df = pd.read_csv('arrhythmia.data', sep = ',', names = col )

In [ ]:
# First 5 rows of the data set.
df.head()

In [ ]:
# Finiding number of rows and columns in the dataset.
rows, columns = df.shape
print('The dataset has {0} rows and {1} columns'.format(rows, columns))

In [ ]:
# Basic statistics analysis of the dataset.
df.describe()

In the preprocessing steps, we will start examining different features, first, we want to investigate the columns that are categorical.

In [ ]:
# Finding all qualatative columns for Training data
qual = [x for x in df.columns if df.dtypes[x] == 'object'] 
print('There are {0} of columns that are categorical'.format(len(qual)))

As we can see there are 5 columns that categorical but we need to do a further investigation if they actually categorical or not. One such procedure is to investigate each feature individually. And one of the easy procedure to find the nature of the categorical variable is to find the numbers of unique values if there a huge number of unique values then we can say that they are not categorical variable but a numeric variable. 

In [ ]:
for x in qual:
    print(df[x].value_counts())     

Interestingly we can see there are a huge number of unique values for each of these variables which is an indication of a numeric variable but we cannot it with certainty, therefore it will be a good idea if we try to understand their nature through some graphs. A boxplot can be a good choice for that. So we will create box plots to see how the data is segregated for different values and we will also replace any null values with the term 'Missing' in the graphs.

In [ ]:
df1 = df.copy()
for v in qual:
    df1[v] = df[v].astype('category')
    if df1[v].isnull().any():
        df1[v] = df1[v].cat.add_categories(['Missing'])
        df1[v] = df1[v].fillna('Missing')

In [ ]:
def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x = plt.xticks(rotation=90)

a = pd.melt(df1, id_vars =['Class'], value_vars =qual )
b = sns.FacetGrid(a, col='variable', col_wrap=3, sharex=False, sharey=False)
b = b.map(boxplot, 'value', 'Class')

From the above pictures and the unique value calculation, we can definitely say none of these categorical variables is categorical rather they are numerical in nature. Therefore, we need to convert these variables to numerical.  

And we also see there are some '?' mark, as values in these variables and we need to treat those values. First, we will remove the column 'J' as it contains mostly '?' marks.
Next, for other columns, we replace the '?' marks will 'null' values then convert those variables into a numeric type and finally will replace those 'null' values with the average value of the variables.

In [ ]:
df.drop(['J'], axis = 1, inplace= True)
print('The column J is removed!!')

In [ ]:
qual = [x for x in df.columns if df.dtypes[x] == 'object']
for x in qual:
    df[x] = df[x].replace(r'?', np.nan)
    df[x] = df[x].astype('float')
    df[x] = df[x].fillna(df[x].mean())

In [ ]:
# Finding all quantitative columns for Training data
qual = [x for x in df.columns if df.dtypes[x] == 'object']

if len(qual) == 0:
    print('There is no categorical variale in the dataset')

Now, we do not have any categorical variables any more. 

Next, we want to see the distrubution plot for all the numerical variable as this will give us the idea about each of the variables. 

In [ ]:
quant = [x for x in df.columns if df.dtypes[x] != 'object']
x = pd.melt(df, value_vars=quant)
x1 = sns.FacetGrid(x, col='variable', col_wrap =4 ,sharex= False, sharey = False)
x1 = x1.map(sns.distplot, 'value')

As we clearly see some of the variables are have a zero values and both 'sex' and 'class' variables are also considered as numerical although they categorical in nature. So we need convert them to categorical type. 

In [ ]:
#Converting to categorical variable for Sex and Class features.
df['Sex'] = df['Sex'].astype('str')
df['Class'] = df['Class'].astype('str')

In [ ]:
df.shape

In [ ]:
df['Sex'].value_counts()

As we can see the 'Sex' is quite proportionate i.e. the number of male and female candidates are close to each other.

Since, we know there is two types of patients, one without arrhythmia and others with arrhythmia. And our target variable has 1 for without arrhythmia patients and then 2 to 15 for different class of arrhythmia and 16 for unclassified arrhythmia. And those with arrhythmia will go the further examination, we can convert the target variable into two categories 1 for without arrhythmia and 0 for with arrhythmia. 

In [ ]:
#df['Class'] = np.where(df['Class'] == '1', 'non-arrhythmia', 'arrhythmia')

In [ ]:
df['Class'] = np.where(df['Class'] == '1', '1', '0')

In [ ]:
df['Class'].value_counts()

As we cn see here, there are 245 patients without arrhythmia and 207 patients with arrhythmia, we can say that the dataset is quite balanced i.e. there will not be any biases due to imbalance target values.

Next, we want to investigate if there is any null values in the dataset.

In [ ]:
for column in df:
    if df[column].isnull().any():
       print('{0} has {1} null values i.e. {2}%.'.format(column, df[column].isnull().sum(), round((df[column].isnull().sum()/rows)*100, 2) ))

    
if df.isnull().any().sum() == 0:
        print('There is no null values in the dataset')

As we can see there is no null value in the dataset we do not need to do any treatment for the null values.

In [ ]:
r, c  = df.shape
print('The dataset has {0} rows and {1} columns'.format(r, c))

Here, we have used Python's scikit-learn library to create and evaluate the machine learning algorithm.  At first, Import of train_test_split function that will help us to split the dataset for training and testing purpose and the accuracy_score function to measure the accuracy of the model. 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# Creating target and feature data
X = df.drop('Class', axis = 1)
y = df['Class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=2)

_As we have said before we will use the Random Forest algorithm to create our machine learning model_

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
print(accuracy_score(y_test, y_pred))

Although we have decided to use Random Forest classifier algorithm to solve our problem, we made an accuracy test for three different algorithms, KNN, Naive Bayes and Random Forest and the calculated accuracy for each of these algorithms and we found Random Forest has better accuracy than other two, so we can consider it as our base model and tune the parameters for this model to achieve higher accuracy. 

_Parameter Tuning_

Here, we have chosen three parameters that are suitable for  Random Forest algorithm that need to tune, the number of trees in the forest which is denoted by n_estimators and we will try different values [5,10, 15, 20, 25, 30, 35, 40], the criteria to split the trees and we have two values 'gini' index or 'entropy' calculation which is denoted by 'criterion'. And finally, the maximum depth of the tree and it is denoted by max_depth in the Random Forest estimator and we will use different values like [1, 2, 3, 4, 5] etc.  

Instead of using any fixed value we will an function called GridSearchCV that will take each of the these values try combination of each of these values then come up with best values for each of these criteria that will give us the best score. It is will also perform a 10 fold cross validation to find best score. 

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = { 'n_estimators':[5, 10, 15, 20, 25, 30, 40],
          'max_depth':[1, 2, 3, 4, 5],
          'criterion':['gini', 'entropy']
          
         }

In [ ]:
grid_rf = GridSearchCV(rf, params,cv = 10, scoring='accuracy')

In [ ]:
grid_rf.fit(X_test,y_test)

In [ ]:
print('The best Score is {:.4f}'.format(grid_rf.best_score_))
print('The best parameters are {0}'.format(grid_rf.best_params_))
print('The best estimator is {0}'.format(grid_rf.best_estimator_))

In [ ]:
y_pred = grid_rf.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_pred))

_d) Evaluate the model you have trained and give a projection for roughly how well you would expect this model to perform after deployment. If you have any suggestions as to how the model might be improved you may note them here. (8 marks)_

For many people, an 98% accuracy rate probably sounds pretty good and the story about the test seemed to encourage a perception of precision. It is speculated that the test could lead to “earlier diagnosis and treatment, and better survival” for individuals with a heart condition.

But before we proceed further we need to define a few terms herein context of our problem.

- True positive(TP): Sick people correctly identified as sick
- False-positive(FP): Healthy people incorrectly identified as sick
- True negative(TN): Healthy people correctly identified as healthy
- False-negative(FN): Sick people incorrectly identified as healthy

**Sensitivity** or _true positive rate_ or _recall_ is defined as the ratio of true positive and all predicted positive values i.e. $\frac{TP}{TP +FN}$, this value shows the percentage of people with arrhythmia reading i.e. who are correctly identified as having the heart condition.

And, **Specificity** or _true negative rate_ is defined as the ratio of actual negatives and all predicted negative i.e. $\frac{TN}{TN +FP}$ this percentage value shows the healthy people who are correctly identified as not having any heart condition.

In the medical field, it is more important to find the case with a medical condition, therefore, any number of patients with heart condition diagnosis with not having heart condition can be fatal and we need to minimise that value. Therefore, having high sensitivity is a good indicator for this situation as we need both high TP and low FN.

We also need to calculate the **precision** or _positive predictive value_ which is the ratio of true positives and combined true and false positives $\frac{TP}{TP + FP}$, in another words Precision measures how many of the samples predicted as positive are actually positive.  

These are another performance indicator that is F-Score which is defined as a measure of a test’s accuracy. The F score is defined as the weighted harmonic mean of the test’s precision and recall. 

$ F-Score = 2.\frac{precision.recall}{precision + recall}$

The F score is used to measure a test’s accuracy since it considers both precision and recall, it is a more reliable measure for the test's performance, the range of F score falls between 0 to 1. 


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
# Creating the confusion matrix to understand the situation
confusion = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n{}".format(confusion))

In [ ]:
print(accuracy_score(y_test, y_pred))
target_names = ['arrhythmia', 'non-arrhythmia']
print(classification_report(y_test, y_pred, target_names = target_names))
#print(classification_report(y_test, y_pred))